# Clustering Algorithm for History selection

In [1]:
import pickle
import os
import pandas as pd
import pyarrow
import pyarrow.parquet as pa

from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, cophenet
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform
import numpy as np
import math

# This flag gives the possibility to run the code in testmode
# i.e. test prints and limited I/O output such that it does not exceed the buffer (e.g. only printing one user) 
TEST = False

## Data exploration
### User histories

In [4]:
# Specify the path to save the clustered user histories
user_articles_info_path = "../DATA/user_history_selection/final_cluster_data.txt"
""" Read in user histories article info, i.e. features to use for clustering provided in the following format:

    {<user_id>:
        'index': {<index to corresponding to the lists below>: <article_id>, ...},  # To map back the enries to an article id
        'topics': [['politics', 'economy'], ...],
        'sentiment': [<sentiment_score>, ...],                                      # Range [0, 1]
        'page_views': [100, ...],
        'time_published': [<'yyyy-MM-dd HH:mm:ss'>,...] }                           # String of datetime

"""
user_articles_info = pickle.load(open(user_articles_info_path, 'rb'))

In [5]:
# TEST PRINT
if TEST:
    K = 1
    res = dict(list(user_articles_info.items())[0: K])
    print(res)

In [6]:
print(len(user_articles_info), "users")

19704 users


In [7]:
if TEST:
    example_user_id = list(user_articles_info.keys())[0]
    print("User", example_user_id)
    print(user_articles_info[example_user_id])

In [8]:
df_ua_info = pd.DataFrame(user_articles_info).T

if TEST:
    display(df_ua_info)

# Compute what the average 'page_views' is for a user given the whole dataset, 
# i.e. the average 'page_views' for whole dataset of the average 'page_views' per user history
count_user_avg_page_views = np.array([])
for page_views in df_ua_info["page_views"]:
    user_avg_page_views = np.nanmean(np.array(page_views)) 
    count_user_avg_page_views = np.append(count_user_avg_page_views, user_avg_page_views)

avg_avg_user_page_views = np.nanmean(count_user_avg_page_views)
print("On average, the average 'page_views' per user is", avg_avg_user_page_views)

On average, the average 'page_views' per user is 110880.13820090744


/tmp/ipykernel_5912/2920062914.py:10: RuntimeWarning: Mean of empty slice
  user_avg_page_views = np.nanmean(np.array(page_views))


## Clustering

In [9]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
from time import sleep
import random
from collections import defaultdict, OrderedDict

# Dictionary containing al the relevant clustered articles per user
user_clustered_articles = {}

# Insert per user its cluster ids and corresponding articles
def insert_user_history_clusters(user_id, df, data_index_to_article_id):
    # Get all possible clusters
    clusters = df.cluster.unique()

    if TEST:
        print("Clusters user", user_id, ":", clusters)

    for c in clusters:
        if TEST:
            print("Cluster:", c)
        
        # Get all articles belonging to cluster c
        clustered_articles = df.loc[df['cluster'] == c].copy()
        # Sort articles history by most recent publishing date
        ca_history = clustered_articles.sort_values(by='time_published', ascending = False)
        
        if TEST:
            display(ca_history)
        
        article_id_indices = list(ca_history.index)

        if TEST:
            print("Cluster article indices df:", article_id_indices)
            
        # For each article in the history save the <article_id> as a key and the <time_published> as value
        for idx in article_id_indices:
            # print(idx)
            article_id = data_index_to_article_id[idx]
            timestamp = df.loc[[idx],['time_published']].values[0]

            if TEST:
                print(timestamp)

            if user_id not in user_clustered_articles:
                user_clustered_articles[user_id] = {}
                
            # When cluster does not yet exist for user
            if c not in user_clustered_articles[user_id]:
                user_clustered_articles[user_id][c] = OrderedDict()
                
            user_clustered_articles[user_id][c][article_id] = timestamp

        if TEST:
            for cluster, articles in user_clustered_articles[user_id].items():
                print("Cluster", cluster)
                for article_id, timestamp in articles.items():
                    print("Article id", article_id, ", timestamp", timestamp)
    
            print(user_clustered_articles[user_id])

if TEST:
    N = 1
else:
    N = len(user_articles_info)
    
pbar = tqdm(list(user_articles_info.keys())[:N])

# Loop through all users
for user_id in pbar:
    if TEST:
        user_id = example_user_id
        
    pbar.set_description("Processing user %s" % user_id)

    # Sample data
    data = user_articles_info[user_id]

    data_index_to_article_id = data['index']

    data = {'topics': data['topics'],
            'sentiment': data['sentiment'],
            'page_views': data['page_views'],
           'time_published': data['time_published']}

    # Number of articles
    n_samples = len(data['page_views'])
    
    if TEST:
        print("Number of articles", n_samples)
        
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Calculate the average of these columns ignoring NaN values
    avg = df[['sentiment', 'page_views', 'time_published']].mean(skipna=True)

    # When average page view is nan take the user's average 'page_view' on average 
    # (this case only occurs when user has only one article for which this value is nan 
    if math.isnan(avg['page_views']):
        avg['page_views'] = avg_avg_user_page_views

    # Imputation by the mean
    df = df.fillna({'sentiment': avg['sentiment'], 'page_views': avg['page_views'], 'time_published': avg['time_published']})

    # User history containing only one article needs no clustering, hardcode data and continue
    if len(df.index) == 1:
        df['cluster'] = [1]
        insert_user_history_clusters(user_id, df, data_index_to_article_id)
        continue
    
    # Multi-Label Binarizer for 'topics'
    mlb = MultiLabelBinarizer()
    # Normalizer for 'page_views'
    scaler = MinMaxScaler()

    topics_encoded = mlb.fit_transform(df['topics'])
    page_views_normalized = scaler.fit_transform(df[['page_views']])
    
    # Convert 'time_published' to datetime and extract the hour
    df['time_published'] = pd.to_datetime(df['time_published'], unit='s', format='%y-%m-%d %H:%M:%S')
    df['hour'] = df['time_published'].dt.hour
    
    # Normalize the hour (0-23) to be between 0 and 1
    hour_normalized = df['hour'] / 23.0
    hour_normalized = hour_normalized.values.reshape(-1, 1)
    
    # Combine all features
    features = np.hstack((topics_encoded, df[['sentiment']].values, page_views_normalized, hour_normalized))
    
    # Define a function to calculate Gower distance
    def gower_distance(X):
        individual_variable_distances = []
        for col in range(X.shape[1]):
            if np.issubdtype(X[:, col].dtype, np.number):
                range_ = np.ptp(X[:, col])
                if range_ == 0:
                    range_ = 1  # avoid division by zero
                individual_variable_distances.append(pdist(X[:, col].reshape(-1, 1), metric='euclidean') / range_)
            else:
                individual_variable_distances.append(pdist(X[:, col].reshape(-1, 1), metric='hamming'))
        return np.sqrt(sum(individual_variable_distances))
    
    # Calculate Gower distance matrix
    gower_dist_matrix = squareform(gower_distance(features))
    
    # Perform hierarchical clustering
    Z = linkage(gower_dist_matrix, method='ward')
    
    # Calculate cophenetic distances
    c, coph_dists = cophenet(Z, pdist(gower_dist_matrix))

    if TEST:
        print("cophenetic coefficient", c)
        print("cophenetic distances", coph_dists)

    if TEST:
        print("Min cophenetic distance threshold =", math.floor(min(coph_dists)))
        print("Max cophenetic distance threshold =", math.floor(max(coph_dists)))

        # Plot the dendrogram
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 7))
        dendrogram(Z)
        plt.title('Hierarchical Clustering Dendrogram')
        plt.xlabel('Sample index')
        plt.ylabel('Distance')
        plt.show()

    # Determine the range of t values from the linkage matrix
    min_distance = math.floor(min(coph_dists))
    max_distance = math.floor(max(coph_dists))

    # Extend the range slightly beyond the min and max distances
    t_values = np.linspace(min_distance + 0.5, max_distance + 0.5, 100)

    # Try different thresholds for parameter t and determine optimal value
    silhouette_scores = []

    for t in t_values:
        clusters = fcluster(Z, t, criterion='distance')
        num_clusters = len(set(clusters))

        if TEST:
            print("Trying value", t)
            print(num_clusters, clusters)
        
        if num_clusters > 1:  # Ensure there is more than one cluster
            if num_clusters >=  n_samples:
                silhouette_scores.append(-1) # Assign a low score for number of clusters not in range [2, n_samples - 1]
            else:
                score = silhouette_score(gower_dist_matrix, clusters)
                silhouette_scores.append(score)
        else:
            silhouette_scores.append(-1)  # Assign a low score for a single cluster
    
    # Find the t value that maximizes the silhouette score
    optimal_t = t_values[np.argmax(silhouette_scores)]

    if TEST:
        print(f"Optimal t value: {optimal_t}")
        
    # Determine cluster assignments
    max_d = optimal_t  # This value could also be set based on the dendrogram
    clusters = fcluster(Z, max_d, criterion='distance')
    
    df['cluster'] = clusters

    if TEST:
        display(df)

    insert_user_history_clusters(user_id, df, data_index_to_article_id)

Processing user 1957332: 100%|██████████| 19704/19704 [5:44:13<00:00,  1.05s/it]


## DATA I/O
### Save clustered data

In [10]:
import pickle

# Write data: clustered user articles history
# Structures :
"""
{<user_id>: 
    {<cluster_id>: 
        {<article_id>: <timestamp_published>,
        <article_id>: <timestamp_published>,
        …,
        <article_id>: <timestamp_published>}
    }
}
"""
with open('../DATA/user_history_selection/user_clustered_articles_history.pickle', 'wb') as handle:
    pickle.dump(user_clustered_articles, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Test: Read in saved data

In [11]:
# Test
with open('../DATA/user_history_selection/user_clustered_articles_history.pickle', 'rb') as handle:
    file = pickle.load(handle)

K = 1
res = dict(list(file.items())[0: K])
print(res)

{'2444624': {130: OrderedDict([('9744055', array(['2023-05-02T03:07:21.000000000'], dtype='datetime64[ns]')), ('9558040', array(['2022-12-25T08:41:59.000000000'], dtype='datetime64[ns]'))]), 111: OrderedDict([('9767557', array(['2023-05-16T16:44:29.000000000'], dtype='datetime64[ns]'))]), 19: OrderedDict([('9764079', array(['2023-05-13T21:47:28.000000000'], dtype='datetime64[ns]')), ('9762836', array(['2023-05-12T21:48:28.000000000'], dtype='datetime64[ns]')), ('9760386', array(['2023-05-11T15:09:38.000000000'], dtype='datetime64[ns]')), ('9752244', array(['2023-05-07T09:38:40.000000000'], dtype='datetime64[ns]')), ('9751220', array(['2023-05-05T17:55:02.000000000'], dtype='datetime64[ns]')), ('9743831', array(['2023-05-01T07:53:29.000000000'], dtype='datetime64[ns]')), ('9742408', array(['2023-04-30T04:08:58.000000000'], dtype='datetime64[ns]')), ('9740400', array(['2023-04-28T10:07:15.000000000'], dtype='datetime64[ns]'))]), 52: OrderedDict([('9765551', array(['2023-05-15T02:13:35.00